In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

In [2]:
mnist = input_data.read_data_sets('MNSIT_data', one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNSIT_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNSIT_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNSIT_data/t10k-images-idx3-ubyte.gz
Extracting MNSIT_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [3]:
#输出准确率
def compute_accuracy(v_xs, v_ys):
    global prediction
    y_pre = sess.run(prediction, feed_dict={xs:v_xs, keep_prob:1})
    correct_prediction = tf.equal(tf.argmax(y_pre,1), tf.argmax(v_ys,1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))  #cast 类型转换成tf.float32
    result = sess.run(accuracy, feed_dict={xs:v_xs, ys:v_ys, keep_prob:1})
    return result

In [4]:
#权重
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1) #正态分布， stddev 标准差， mean 均值
    return tf.Variable(initial)

In [5]:
#偏置
def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape) #常量张量
    return tf.Variable(initial)

In [6]:
#卷积层
def conv2d(x,W): #x=input value, W=weight, strides[0] = strides[3] = 1
    return tf.nn.conv2d(x,W, strides=[1,1,1,1], padding='SAME') #strides [1,x_movement,y_movement,1]

In [7]:
#pooling
def max_pool_2x2(x): #池化一般取2*2中最大的数，不够2*2以0填充
    return tf.nn.max_pool(x, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME') #ksize = kernel size

In [8]:
#define placehoder for inputs to network
xs = tf.placeholder(tf.float32,[None, 784]) #28*28
ys = tf.placeholder(tf.float32,[None,10])
keep_prob = tf.placeholder(tf.float32)

In [9]:
x_image = tf.reshape(xs, [-1,28,28,1])  # -1,暂时不用管维度大小    1，channel，黑白图片故只有1
#print(x_image.shape)

In [10]:
#conv1 layer
W_conv1 = weight_variable([5,5,1,32])#patch 5*5, in size 1 - image depth, out size 32 - extract 32 feature
b_conv1 = bias_variable([32])
h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1) #output size = 28*28*32
h_pool1 = max_pool_2x2(h_conv1) #output size = 14*14*32

In [11]:
#conv2 layer
W_conv2 = weight_variable([5,5,32,64])#patch 5*5, in size 32 - image depth, out size 64 - extract 64 feature
b_conv2 = bias_variable([64])
h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2) #output size = 14*14*64
h_pool2 = max_pool_2x2(h_conv2)  #output size = 7*7*64

In [12]:
#func1 layer
W_fc1 = weight_variable([7*7*64,1024])
b_fc1 = bias_variable([1024])
h_pool2_flat = tf.reshape(h_pool2,[-1,7*7*64]) #[n_samples,7,7,64] ->> [n_samples,7*7*64]
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1)+b_fc1)
#dropout
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

In [13]:
#fun2 layer
W_fc2 = weight_variable([1024,10])
b_fc2 = bias_variable([10])

prediction = tf.nn.softmax(tf.matmul(h_fc1_drop, W_fc2)+b_fc2)

In [14]:
#loss == the error between prediction and real data
cross_entropy = tf.reduce_mean(-tf.reduce_sum(ys*tf.log(prediction), reduction_indices=[1])) #loss  -> tf.nn.softmax

In [15]:
train_step = tf.train.AdadeltaOptimizer(0.05).minimize(cross_entropy) #learning_rate -> 1e-4 = 0.0001

In [16]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())
for i in range(10000):
        batch_xs, batch_ys = mnist.train.next_batch(100) #从mnist 提出100
        sess.run(train_step, feed_dict={xs:batch_xs,ys:batch_ys, keep_prob:0.6})
        if i%50 == 0:
            print(compute_accuracy(mnist.test.images, mnist.test.labels))

0.112
0.3537
0.5572
0.6874
0.7567
0.7988
0.8276
0.8521
0.8658
0.8761
0.8878
0.8966
0.9044
0.9077
0.9154
0.9189
0.9219
0.9266
0.9282
0.931
0.934
0.9353
0.9374
0.9392
0.9419
0.9434
0.9433
0.9445
0.9455
0.9472
0.9491
0.9492
0.9502
0.9507
0.9504
0.9518
0.9542
0.9534
0.9549
0.9555
0.9548
0.956
0.9565
0.9564
0.9572
0.9575
0.9586
0.9591
0.9597
0.9593
0.9598
0.9621
0.9613
0.9615
0.9618
0.9621
0.9635
0.9623
0.9629
0.9638
0.9641
0.9649
0.964
0.9667
0.9664
0.9651
0.9667
0.9668
0.9676
0.966
0.9673
0.9665
0.9667
0.9683
0.968
0.9677
0.9694
0.9693
0.9689
0.9691
0.9702
0.9696
0.9697
0.9708
0.9704
0.9701
0.9712
0.9702
0.9704
0.9718
0.971
0.9722
0.9726
0.9723
0.9717
0.9731
0.9734
0.9729
0.9723
0.9734
0.9731
0.9729
0.9731
0.9736
0.9732
0.9736
0.9736
0.9739
0.974
0.9749
0.9756
0.975
0.975
0.9747
0.9758
0.9747
0.976
0.9754
0.9757
0.9755
0.9757
0.9751
0.9754
0.9763
0.9758
0.9754
0.9755
0.9759
0.9764
0.9765
0.9763
0.9766
0.9765
0.9775
0.9776
0.9776
0.9775
0.9774
0.9777
0.9769
0.9776
0.9776
0.9784
0.9783
0.97